In [59]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics import mean_squared_error

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.callbacks import EarlyStopping

In [60]:
total = pd.read_csv("../CSV/train_data.csv")
test = pd.read_csv("../CSV/pred_test.csv")

print("total shape : {}, test shape : {}".format(total.shape, test.shape))

total shape : (368088, 12), test shape : (15120, 12)


In [61]:
def nmae(true_df, pred_df):
    target_idx = true_df.iloc[:,0]
    pred_df = pred_df[pred_df.iloc[:,0].isin(target_idx)]
    pred_df = pred_df.sort_values(by=[pred_df.columns[0]], ascending=[True])
    true_df = true_df.sort_values(by=[true_df.columns[0]], ascending=[True])
    
    true = true_df.iloc[:,1].to_numpy()
    pred = pred_df.iloc[:,1].to_numpy()
    
    score = np.mean((np.abs(true-pred))/true)
    
    return score

In [62]:
train_years = [2013,2014,2015,2016,2017]
val_years = [2018]

train = total[total['year'].isin(train_years)]
val = total[total['year'].isin(val_years)]

# "year", "month", "day", "hour", "weekday", "구분_int", "기온(°C)"
features = ["month", "hour", "구분_int", "기온(°C)"]
# features = ["year", "month", "day", "hour", "weekday", "구분_int", "기온(°C)"]
train_x = train[features]
train_y = train['공급량']

val_x = val[features]
val_y = val['공급량']

X = total[features]
y = total["공급량"]

In [63]:
print(train_x.shape, train_y.shape)
print(val_x.shape, val_y.shape)
print(test.shape)

(306768, 4) (306768,)
(61320, 4) (61320,)
(15120, 12)


In [64]:
model = Sequential()
model.add(Dense(units = 16, input_dim = 4, activation = "relu"))
model.add(Dense(32, activation = "relu"))
model.add(Dense(64, activation = "relu"))
model.add(Dense(32, activation = "relu"))
model.add(Dense(16, activation = "relu"))
model.add(Dense(1))

model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_30 (Dense)            (None, 16)                80        
                                                                 
 dense_31 (Dense)            (None, 32)                544       
                                                                 
 dense_32 (Dense)            (None, 64)                2112      
                                                                 
 dense_33 (Dense)            (None, 32)                2080      
                                                                 
 dense_34 (Dense)            (None, 16)                528       
                                                                 
 dense_35 (Dense)            (None, 1)                 17        
                                                                 
Total params: 5,361
Trainable params: 5,361
Non-traina

In [65]:
model.compile(loss = "mean_squared_error", 
              optimizer = "adam", 
              metrics = ["accuracy"])

early_stopping = EarlyStopping(patience = 30)

hist = model.fit(train_x, train_y, 
                 epochs = 30, 
                 batch_size = 10, 
                 validation_data = (val_x, val_y), 
                 callbacks = [early_stopping])

Epoch 1/30
30677/30677 [==============================] - 29s 933us/step - loss: 386389.7812 - accuracy: 0.0000e+00 - val_loss: 62305.5820 - val_accuracy: 0.0000e+00
Epoch 2/30
30677/30677 [==============================] - 28s 907us/step - loss: 44411.4375 - accuracy: 0.0000e+00 - val_loss: 40457.8594 - val_accuracy: 0.0000e+00
Epoch 3/30
30677/30677 [==============================] - 27s 869us/step - loss: 36116.2227 - accuracy: 0.0000e+00 - val_loss: 36444.6211 - val_accuracy: 0.0000e+00
Epoch 4/30
30677/30677 [==============================] - 26s 842us/step - loss: 32767.9805 - accuracy: 0.0000e+00 - val_loss: 31970.9668 - val_accuracy: 0.0000e+00
Epoch 5/30
30677/30677 [==============================] - 26s 832us/step - loss: 29567.9434 - accuracy: 0.0000e+00 - val_loss: 36210.7266 - val_accuracy: 0.0000e+00
Epoch 6/30
30677/30677 [==============================] - 26s 844us/step - loss: 28041.0195 - accuracy: 0.0000e+00 - val_loss: 39026.2031 - val_accuracy: 0.0000e+00
Epoch 7/3

In [66]:
model.evaluate(val_x, val_y)

1917/1917 [==============================] - 2s 780us/step - loss: 28709.6562 - accuracy: 0.0000e+00


[28709.65625, 0.0]

In [67]:
pred = model.predict(val_x)

true_y = pd.DataFrame({"y" : val_y})
pred_y = pd.DataFrame(pred)

true_y.reset_index(drop = True, inplace = True)
true_y.reset_index(inplace = True)
pred_y.reset_index(inplace = True)

print("NMAE : {:.6f}".format(nmae(true_y, pred_y)))

NMAE : 0.551136


In [56]:
# pred_y
# true_y

In [57]:
submission = pd.read_csv('../CSV/가스공급량 수요예측 모델개발 data/sample_submission.csv')

In [58]:
test_x = test[features]
preds = model.predict(test_x)
submission['공급량'] = preds
submission.to_csv("../CSV/Submission/27_4_features_DeepLearning_model.csv", index = False)